In [ ]:
# gui_ann_FF_full.py
import json
import numpy as np
import tkinter as tk
from tkinter import ttk, messagebox, filedialog

MODEL_JSON = "ann_FF_model.json"

def tanh(x):
    return np.tanh(x)

def forward_pass(x_row, W1, b1, W2, b2):
    W1 = np.array(W1)
    b1 = np.array(b1).reshape(1, -1)
    W2 = np.array(W2)
    b2 = np.array(b2).reshape(1, -1)
    
    h = tanh(x_row @ W1 + b1)
    y = h @ W2 + b2
    return float(y[0,0])

def scale_minmax_row(x_row, xmin, xmax, a=-1.0, b=1.0):
    denom = (xmax - xmin)
    denom[denom == 0] = 1.0
    xn = (x_row - xmin) / denom
    return a + xn * (b - a)

class CSApp:
    def __init__(self, root):
        self.root = root
        self.root.title("ANN-FF Compressive Strength Predictor")
        self.root.geometry("600x700")
        self.model = None

        # Top frame with model info and load button
        frm_top = ttk.Frame(root, padding=10)
        frm_top.pack(fill="x")
        self.model_label = ttk.Label(frm_top, text=f"Model: {MODEL_JSON}")
        self.model_label.pack(side="left")
        btn_load = ttk.Button(frm_top, text="Load Model...", command=self.load_model_dialog)
        btn_load.pack(side="right")

        # Canvas for title and developer info
        self.canvas1 = tk.Canvas(root, height=180)
        self.canvas1.pack(fill="x")
        self.draw_info_canvas()

        # Scrollable frame for inputs
        frm_inputs = ttk.Frame(root, padding=10)
        frm_inputs.pack(fill="both", expand=True)
        self.canvas = tk.Canvas(frm_inputs)
        scrollbar = ttk.Scrollbar(frm_inputs, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ttk.Frame(self.canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )
        self.canvas.create_window((0,0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=scrollbar.set)
        self.canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        self.entries = []

        # Bottom frame with predict button and result
        frm_bottom = ttk.Frame(root, padding=10)
        frm_bottom.pack(fill="x")
        self.btn_predict = ttk.Button(frm_bottom, text="Predict CS", command=self.predict)
        self.btn_predict.pack(side="left")
        self.lbl_result = ttk.Label(frm_bottom, text="Predicted CS: -- MPa", font=("Arial", 12, "bold"))
        self.lbl_result.pack(side="right")

        self.load_model(MODEL_JSON)

    def draw_info_canvas(self):
        title = "Prediction of CS of Heavy Weight Concrete Using ANN-FF Algorithm"
        self.canvas1.create_text(300, 20, text=title, font=('Futura Md Bt', 13, 'bold'), fill='black')

        developers = [
            "Developed by:",
            "1. Orapan Saensuk, Thammasat University",
            "2. Divesh Ranjan Kumar, Thammasat University",
            "3. Peem Nuaklong, Thammasat University"
        ]
        y = 50
        for dev in developers:
            self.canvas1.create_text(20, y, anchor="w", text=dev, font=('Futura Md Bt', 11), fill='black')
            y += 25

        self.canvas1.create_text(20, y, anchor="w", text="Contact Email: ranjandivesh453@gmail.com", font=('Futura Md Bt', 11), fill='black')
        y += 30
        self.canvas1.create_text(20, y, anchor="w", text="Input Parameters", font=('Futura Md Bt', 12, 'bold', 'italic', 'underline'), fill='black')

    def load_model_dialog(self):
        path = filedialog.askopenfilename(filetypes=[("JSON", "*.json")])
        if path:
            self.load_model(path)

    def load_model(self, path):
        try:
            with open(path, "r") as f:
                self.model = json.load(f)
            self.model_label.config(text=f"Model: {path}")
            self.build_inputs()
        except Exception as e:
            messagebox.showerror("Load Error", f"Could not load model: {e}")

    def build_inputs(self):
        for w in self.scrollable_frame.winfo_children():
            w.destroy()
        self.entries.clear()

        # Feature names with units
        features = [
            ("OPC", "kg/m³"),
            ("MK", "kg/m³"),
            ("Water", "kg/m³"),
            ("Sand", "kg/m³"),
            ("Granite Waste", "kg/m³"),
            ("Coarse Aggregate", "kg/m³"),
            ("Barite", "kg/m³"),
            ("Superplasticizer", "kg/m³"),
            ("w/c Ratio", ""),
            ("Age", "Days")
        ]

        for i, (name, unit) in enumerate(features):
            label_text = f"{i+1}. {name} ({unit})" if unit else f"{i+1}. {name}"
            ttk.Label(self.scrollable_frame, text=label_text, font=("Arial", 10)).grid(row=i, column=0, sticky="w", padx=5, pady=4)
            entry = ttk.Entry(self.scrollable_frame, width=16)
            entry.grid(row=i, column=1, sticky="w", padx=5, pady=4)
            self.entries.append(entry)

    def predict(self):
        if self.model is None:
            messagebox.showerror("No Model", "Load a model JSON first.")
            return

        try:
            x = np.array([float(e.get()) for e in self.entries], dtype=float).reshape(1, -1)
        except ValueError:
            messagebox.showerror("Input Error", "Please enter numeric values in all fields.")
            return

        # Min-max scaling if scaler info exists
        if "scaler" in self.model:
            xmin = np.array(self.model["scaler"]["xmin"], dtype=float)
            xmax = np.array(self.model["scaler"]["xmax"], dtype=float)
            a, b = self.model["scaler"].get("scale_range", [-1.0, 1.0])
            x = scale_minmax_row(x, xmin, xmax, a, b)

        W1 = np.array(self.model["weights"]["W1"], dtype=float)
        b1 = np.array(self.model["weights"]["b1"], dtype=float)
        W2 = np.array(self.model["weights"]["W2"], dtype=float)
        b2 = np.array(self.model["weights"]["b2"], dtype=float)

        y = forward_pass(x, W1, b1, W2, b2)
        self.lbl_result.config(text=f"Predicted CS: {y:.2f} MPa")

def main():
    root = tk.Tk()
    app = CSApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
